In [43]:
# GENETIC ALGORITHMS Pseudo - Code
# =================================
# 1. Initial Population
# 2. Evaluating the population
# 3. Stopping criteria
#   3.1. Selecting parents
#   3.2. Cross Over
#   3.3. Mutation
#   3.3. Evaluating the population
#   3.4. Define surviving population
#   3.5. Go to step 3
# 4. List the best individuals
# ==================================

In [44]:
class Product():
    """A Product Class for all the products' information
    """
    def __init__(self, name, space, price):
        self.name = name
        self.space = space
        self.price = price


In [45]:
from random import random
class Individual():
    # Each individual is a solution. 
    # It contains a combination of the products that can be selected based on the criteria
    # we can initialize a population composed of several individuals. 
    def __init__(self, spaces, prices, space_limit, generation = 0):
        self.spaces = spaces
        self.prices = prices
        self.space_limit = space_limit
        self.generation = generation # the very first generation
        self.chromosome = [] # list of solutions. 0s and 1s.
        
        # Initializing a random chromosome
        for i in range(len(spaces)):
            if random() >= 0.5:
                value = '1' # 1 means we are taking this particular product
            else:
                value = '0' # 0 means we are leaving this particular product
            self.chromosome.append(value)
        # print(self.chromosome)
        return None
    
    def fitness_function(self):
        """ Quality measurement to find out how well the chromosome solves the problem.
            This gives whether the solution can be used for evolution or not. 
            we need to add the spaces of all the 1s in chromosome and leave 0s.

        Returns:
            None : It just calculates the total_space and total_price and saves the variables. 
        """

        self.total_price = 0
        self.total_space = 0
        for i in range(len(self.chromosome)):
            if self.chromosome[i] == '1':
                self.total_space += self.spaces[i] # adding all the spaces
                self.total_price += self.prices[i] # adding all the prices
            if self.total_space > self.space_limit:
                # if the space is greated than the total space
                self.total_price = 0 # we cannot include any product 

        # print(self.total_space)
        # print(self.total_price)
        return None
    
    def crossover(self, parent2):
        """cutting a random gene and crossing over two parents giving two different children

        Args:
            parent2 (Individual): The second parent with which the first parent has to be crossed over.

        Returns:
            list: the list of children as Individuals
        """
        # the point where we need to cut
        cutoff_point = round(random() * len(self.chromosome))

        # Get the chromosome of parent 2 and cut using the cutoff_point for crossing over with parent 1
        child1_chromosome = parent2.chromosome[0 : cutoff_point] + self.chromosome[cutoff_point ::]
        child2_chromosome = self.chromosome[0 : cutoff_point] + parent2.chromosome[cutoff_point ::]

        # we need to create new individuals for the born children.
        # Note that we are adding 1 to the generation. This denotes the next generation. 
        child1_individual = Individual(self.spaces, self.prices, self.space_limit, self.generation + 1)
        child2_individual = Individual(self.spaces, self.prices, self.space_limit, self.generation + 1)

        children = [child1_individual, child2_individual]

        children[0].chromosome = child1_chromosome # We need to assign the chromosome to the individual children now
        children[1].chromosome = child2_chromosome

        return children

    def mutation(self, rate):
        """ we just randomly change one of the genes in the individuals.
            for example: making 1 into 0 and vice versa.
            we are not creating anything new here.
            we don't apply this very often

        Args:
            rate (float): The rate of mutation is specified here

        Returns:
            list: The mutated chromosome is returned
        """
        # 
        for i in range(len(self.chromosome)): # it will go through each one of the genes
            if random() < rate:
                if self.chromosome[i] == '1':
                    self.chromosome[i] == '0'
                else:
                    self.chromosome[i] == '1'
        return self # returning the self. 
        

In [46]:
class GeneticAlgorithm():
    def __init__(self, population_size):
        self.population_size = population_size
        self.population = [] # we are going to store all individual objects here. 
        # There will be individual objects in each position of this list
        self.generation = 0 # the current generation
        self.best_solution = None # we will store the best individual considering all generations
        self.list_of_solutions = []
    
    def initialize_population(self, spaces, prices, space_limit):
        # First step of the Genetic Algorithms
        for i in range(self.population_size):
            # we are calling the individual class constructor here
            self.population.append(Individual(spaces = spaces, prices = prices, space_limit = space_limit))
        self.best_solution = self.population[0] # just a random position to initialize the best solution. 
        return None
    
    def order_population(self):
        """ We need to order the population in decending order.
            If we take the 0th position, we pick the one with highest total value
            Note - We need to call this function after getting the fitness function.
            Each item population is an object of Individual class
            We need to put the first position in this list as self.best_solution
        """
        self.population = sorted(self.population, key = lambda population: population.total_price, reverse = True)
        print(self.population)
        return None
    
    def best_individual(self, individual):
        """We need to put the first position in this list as self.best_solution

        Args:
            individual (Individual class): _description_
        """
        if individual.total_price > self.best_solution.total_price:
            self.best_solution = individual
        return None
    
    def sum_evaluations(self):
        """ we are getting all the prices of each individual
            this sum is very useful for selecting parents using roulette method

        Returns:
            float: this function will go through each individual and return the sum of all the prices.
        """
        sum = 0
        for individual in self.population:
            sum += individual.total_price
        return sum

    def select_parents(self, sum_evaluation):
        # how to select the best individuals to create the new generation children
        # There are many methods. However, Rouelette method is very common. 
        # The idea is to pick 2 from the population list and perform fitness, cross over, mutation. 
        parent = -1 # index stars with 0 in python. So we need to use -1 if we didn't select anything 
        random_value = random() * sum_evaluation
        sum = 0
        i = 0
        while i < len(self.population) and sum < random_value:
            # this loop will surely run for the length of the population.
            # the second condition is important. The random value is the total probability of selecting the best parent using roulette method. 
            sum += self.population[i].total_price
            parent += 1
            i += 1
        return parent

    def visualize_generations(self):
        # just printing the generations for viewing purpose
        best = self.population[0]
        print('Generation: ', self.population[0].generation,
          'Total price: ', best.total_price, 'Space: ', best.total_space,
          'Chromosome: ', best.chromosome)
        return None
    
    def genetic_algorithm_complete(self, mutation_probability, num_of_generations, spaces, prices, space_limit):
        # runs all the steps in genetic algorithm
        # 1. Initialize 
        self.initialize_population(spaces, prices, space_limit)
        # 2. Evaluate the initial population
        for individual in self.population:
            individual.fitness_function()
        self.order_population()
        # 3. stopping criteria
        for generation in range(num_of_generations):
            # 3.1 select parents
            sum = self.sum_evaluations()
            new_population = []
            for new_individuals in range(0, self.population_size, 2):
                parent_1 = self.select_parents(sum)
                parent_2 = self.select_parents(sum)
                # 3.2 cross over
                children = self.population[parent_1].crossover(self.population[parent_2])
                # 3.3 mutation
                new_population.append(children[0].mutation(mutation_probability))
                new_population.append(children[1].mutation(mutation_probability))

            self.population = list(new_population)
            # 3.3 Evaluate the population again
            for individual in self.population:
                # evaluating individual children
                # print(individual)
                individual.fitness_function()
        self.visualize_generations()
        # getting the best individual
        best = self.population[0]
        self.best_individual(best)

        print('*** Best Generation ***')
        print('Generation: ', self.population[0].generation,
          'Total price: ', best.total_price, 'Space: ', best.total_space,
          'Chromosome: ', best.chromosome)
        
        # we are returning the chromosome of the best solution
        return self.best_solution.chromosome


In [47]:
products_list = []
products_list.append(Product('Refrigerator A', 0.751, 999.90))
products_list.append(Product('Cell phone', 0.00000899, 2199.12))
products_list.append(Product('TV 55', 0.400, 4346.99))
products_list.append(Product("TV 50' ", 0.290, 3999.90))
products_list.append(Product("TV 42' ", 0.200, 2999.00))
products_list.append(Product("Notebook A", 0.00350, 2499.90))
products_list.append(Product("Ventilator", 0.496, 199.90))
products_list.append(Product("Microwave A", 0.0424, 308.66))
products_list.append(Product("Microwave B", 0.0544, 429.90))
products_list.append(Product("Microwave C", 0.0319, 299.29))
products_list.append(Product("Refrigerator B", 0.635, 849.00))
products_list.append(Product("Refrigerator C", 0.870, 1199.89))
products_list.append(Product("Notebook B", 0.498, 1999.90))
products_list.append(Product("Notebook C", 0.527, 3999.00))

# Keeping all spaces, prices, names in list. 
# We need to deal with the index values if we want to take something out
spaces = []
prices = []
names = []
space_limit = 3 # The stopping criteria

for product in products_list:
    spaces.append(product.space)
    prices.append(product.price)
    names.append(product.name)

population_size = 20
mutation_probability = 0.01
num_of_generations = 100

In [48]:
ga = GeneticAlgorithm(population_size = population_size)
result = ga.genetic_algorithm_complete(mutation_probability = mutation_probability, num_of_generations = num_of_generations, spaces = spaces, prices = prices, space_limit = space_limit)

[<__main__.Individual object at 0x000001BCD095A3E0>, <__main__.Individual object at 0x000001BCD095A0E0>, <__main__.Individual object at 0x000001BCD0963850>, <__main__.Individual object at 0x000001BCD09621D0>, <__main__.Individual object at 0x000001BCD0961ED0>, <__main__.Individual object at 0x000001BCD0961BD0>, <__main__.Individual object at 0x000001BCD095AA70>, <__main__.Individual object at 0x000001BCD0961E70>, <__main__.Individual object at 0x000001BCD0961810>, <__main__.Individual object at 0x000001BCD095B820>, <__main__.Individual object at 0x000001BCD0AD9810>, <__main__.Individual object at 0x000001BCD09630D0>, <__main__.Individual object at 0x000001BCD0962050>, <__main__.Individual object at 0x000001BCD095AF20>, <__main__.Individual object at 0x000001BCD095BEB0>, <__main__.Individual object at 0x000001BCD0962170>, <__main__.Individual object at 0x000001BCD095B670>, <__main__.Individual object at 0x000001BCD0960DF0>, <__main__.Individual object at 0x000001BCD0962AD0>, <__main__.I